In [89]:
import pandas as pd
from pathlib import Path

In [90]:
SOURCE = Path("xlsx/norte/ambientes_salto.xlsx")

In [91]:
df = pd.read_excel(SOURCE, skiprows=1, usecols="B:Z")

/home/bossa/.local/share/virtualenvs/pythonSDK-zBAoyeeM/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [92]:
df = df[~df.Código.isna()]

In [93]:
df.drop(columns=df.columns[15:], inplace=True)

In [94]:
def rewrite_columns(string):
    replacing = [(" ", "_"), ("ã", "a"), ("á", "a"), ("õ", "o"), ("²", ""), ("ç", "c"), ("é", "e"), ("í", "i"), ("ó", "o"), ("ú", "u"), ("(", ""), (")", ""), ("%", "")]
    for pair in replacing:
        string = string.replace(pair[0], pair[1])
    return string.lower()


df.columns = [rewrite_columns(x) for x in df.columns]

In [95]:
ambientes_origem = [x for x in df.columns if 'ambiente_origem' in x]

df["origem"] =  df[ambientes_origem].apply(lambda x: [ amb.split(' - ')[0] for amb in x.values if type(amb) == str ], axis=1)

df.drop(columns=ambientes_origem, inplace=True)

In [96]:
df

,construcao,pavimento,ala,tipo_de_ambiente,setor_responsavel,nomenclatura,nome,descricao,observacoes,codigo,sigla,dimensao_m,origem
0,Bloco A,Térreo,Direita,Corredor,Direção Geral,Corredor D1 Bloco A,Corredor Direito no Bloco A 01,NaN,NaN,ACD1,NaN,148.47,[BC00]
1,Bloco A,Térreo,Frontal,Corredor,Direção Geral,Hall F1 Bloco A,Hall de Entrada Frontal no Bloco A 01,NaN,NaN,ACF1,NaN,31.17,[BC00]
2,Bloco A,Térreo,Direita,Corredor,Direção Geral,Hall D2 Bloco A,Hall de Entrada Direito no Bloco A 02,NaN,NaN,ACD2,NaN,37.92,[BC00]
3,Bloco A,Térreo,Central,Corredor,Direção Geral,Corredor C3 Bloco A,Corredor Central no Bloco A 03,NaN,NaN,ACC3,NaN,34.74,[BC00]
4,Bloco A,Térreo,Esquerda,Corredor,Direção Geral,Corredor E1 Bloco A,Corredor Esquerdo no Bloco A 01,NaN,NaN,ACE1,NaN,125.10,[BC00]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,Bloco T,Andar 1,Direita,Auditório,Direção Administrativa,Platéia DS,Platéia Direita Superior,NaN,NaN,T101,NaN,NaN,[]
257,Bloco T,Andar 1,Esquerda,Auditório,Direção Administrativa,Platéia ES,Platéia Esquerda Superior,NaN,NaN,T103,NaN,NaN,[]
258,Bloco T,Andar 1,Central,Apoio,Direção Administrativa,Mezanino Teatro,Mezanino do Teatro,NaN,NaN,T102,NaN,NaN,[]
259,Bloco T,Andar 1,Central,Apoio,Direção Administrativa,Cabine Teatro,Cabine de Audiovisual do Teatro,NaN,NaN,T102.1,NaN,NaN,[]


In [97]:
dicionarios = df.to_dict('records')

In [98]:
registros = []

for item in dicionarios:
    key = item.pop('codigo')
    nome = item.pop('nome')
    origem = item.pop('origem')
    registros.append({
        'nome': nome,
        'origem': origem,
        'codigo': key,
        'detalhes':  item})

In [99]:
import json 

OUTPUT_PATH = Path("json") / "ambientes-norte.json"

with open(OUTPUT_PATH, "w") as fP:
    json.dump(registros, fP, indent=2, ensure_ascii=False)